In [1]:
%%capture
%pip install - r requirement.txt
# python==3.6.13

In [2]:
import numpy as np
import pandas as pd
import timeout_decorator
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpStatus, value
import time

debug = 1  # debug mode
TIMEOUT = 10 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns
# randomly choose n samples
n = 20000

### PARAMETER INPUT

`None` stands for no boundary.

Hereby its just a demo.

In [3]:
# boolen
NbvCost = True  # True: max nbv; False: min cost

# float
minTotalNbv = 20000000
maxTotalNbv = 800000000

# float
minTotalCost = 20000000
maxTotalCost = 100000000

# float
fleetAgeLowBound = [None, 3, 4]
fleetAgeUpBound = [3, 6, 10]
fleetAgeLimit = [0.2, 0.3, 0.3]

# float
OnHireLimit = 1.0

# float
weightedAgeLowBound = [None, 3, 4]
weightedAgeUpBound = [4, 6, 15]
weightedAgeLimit = [0.1, 0.1, 0.2]

# string
productType = ['D20', 'D4H', 'R4H']
# float
productLimit = [0.1, 0.5, 0.0]

# string
lesseeType = ['MSC', 'ONE', 'HAPAG']
# float
lesseeLimit = [0.6, 0.3, 0.2]

# string
contractType = ['LT', 'LE', 'LF']
# float
contractLimit = [0.3, 0.11, 0.15]

### READ DATA
TODO: read data

In [4]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')
rawData = rawData.iloc[:30000, :61]

In [5]:
data = rawData.sample(n)
print(data.shape)

(20000, 61)


In [6]:
if debug:
    print(data['Product'].value_counts())
    print(data['Billing Status Fz'].value_counts())
    print(data['Contract Cust Id'].value_counts())
    print(data['Contract Lease Type'].value_counts())

D4H    15721
D20     4279
Name: Product, dtype: int64
ON    20000
Name: Billing Status Fz, dtype: int64
MSC         8426
ONE         7333
HAPAG       3859
TCLC         127
COSMR        113
CMA           81
PANOCEAN      61
Name: Contract Cust Id, dtype: int64
LT    11145
LE     4996
LF     3859
Name: Contract Lease Type, dtype: int64


### DATA PREPARATION

TODO: change the column name 

#### Container Age

Assign to `FleetAge1`, `FleetAge2`, `FleetAge3`

In [7]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
        
    if fleetAgeLowBound[i] <= age < fleetAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['FleetAge1'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 0), axis=1)
data['FleetAge2'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 1), axis=1)
data['FleetAge3'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 2), axis=1)

#### Status

Assign new column `Status`

In [8]:
def SelectStatus(status):
    # no need to check input
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

data['Status'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

#### Weighted Age

Assign new column `WeightedAge1`, `WeightedAge2`, `WeightedAge3`

In [9]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
        
    if weightedAgeLowBound[i] <= age < weightedAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['WeightedAge1'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 0), axis=1)
data['WeightedAge2'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 1), axis=1)
data['WeightedAge3'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 2), axis=1)

#### Product

Assign new column `ProductType1`, `ProductType2`, `ProductType3`

In [10]:
def SelectProductType(product, i):
    if product == productType[i]:
        product = 1
    else:
        product = 0
    return product

data['ProductType1'] = data.apply(lambda x: SelectProductType(x['Product'], 0), axis=1)
data['ProductType2'] = data.apply(lambda x: SelectProductType(x['Product'], 1), axis=1)
data['ProductType3'] = data.apply(lambda x: SelectProductType(x['Product'], 2), axis=1)

#### Lessee

Assign new column `Lessee1`, `Lessee2`, `Lessee3`

In [11]:
def SelectLessee(lessee, i):
    if lessee == lesseeType[i]:
        lessee = 1
    else:
        lessee = 0
    return lessee

data['Lessee1'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 0), axis=1)
data['Lessee2'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 1), axis=1)
data['Lessee3'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 2), axis=1)

#### Contract Type

Assign new column `ContractType1`, `ContractType2`, `ContractType3`

In [12]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

data['ContractType1'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 0), axis=1)
data['ContractType2'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 1), axis=1)
data['ContractType3'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 2), axis=1)

### SAVE DATA

In [13]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [14]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv')
data.sample(3)

,Unnamed: 0,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
17116,5015,FZOPS,DF3RM,FDCU0043656,3682.0,DFIC-FL-1801,2/28/2018,1121049.0,DFC3,D4H,FDCU0043656,Dong Fang,2.0,1.7,MSC,LT-MSC-35B,2018-03-01,2032-07-31,LT,USD,1.0,1.07,1.07,ST,0.39,0.39,0.0,-15.42,0.674,2933.008,AC,ON,GO,RLI,AVA,4.33,NaN,I,FUSBI,10.095890,7.361,17.163014,2018.0,F - OL / < 8 yrs,10.095890,17.163014,0.0,2032.0,7.0,14.425890,FCI ANZ 100M,FCI,1.7,2018 NP,Reasonable,8.637164,15.886849,from ANZ portfolio,LT-MSC-35BD4H,NaN,5621.0,1.0,0,1,1,1,0,0,1,0,1,0,1,0,0,1,0,0
999,14573,FZOPS,DF3RM,FFAU4040621,6000.0,CSIH-FL-2115,4/8/2021,1706959.0,DFC3,D4H,FFAU4040621,Dong Fang,2.0,1.7,MSC,LT-MSC-48,2021-04-01,2029-06-30,LT,USD,1.0,1.95,1.95,FR,1.95,1.95,720.0,-27.69,0.000,5612.350,AC,ON,GO,RLI,AVA,1.23,NaN,I,FUSBI,7.008219,2.091,11.913973,2021.0,F - FL / < 8 yrs,7.008219,11.913973,0.0,2029.0,6.0,8.238219,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.367164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0
5471,24840,FZOPS,DF3RM,FBIU0354023,2267.0,CIMC-FL-1804,3/22/2018,1185748.0,DFC3,D20,FBIU0354023,Dong Fang,1.0,1.0,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,0.69,0.69,FA,0.69,0.69,0.0,-8.27,0.618,1880.707,AC,ON,GO,RLI,AVA,4.27,NaN,I,FUSBI,4.295890,4.270,4.295890,2018.0,F - OL / < 8 yrs,4.295890,4.295890,0.0,2026.0,10.0,8.565890,CNIC,FCI,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,1,0


### Model Part

#### Prepare Parameters

convert to numpy

In [15]:
nbv = data['Nbv'].to_numpy()
cost = data['Cost'].to_numpy()
fleetAge = np.stack([data['FleetAge1'].to_numpy(),
                     data['FleetAge2'].to_numpy(),
                     data['FleetAge3'].to_numpy()])
status = data['Status'].to_numpy()
weightedAge = np.stack([data['WeightedAge1'].to_numpy(),
                        data['WeightedAge2'].to_numpy(),
                        data['WeightedAge3'].to_numpy()])
product = np.stack([data['ProductType1'].to_numpy(),
                        data['ProductType2'].to_numpy(),
                        data['ProductType3'].to_numpy()])
lessee = np.stack([data['Lessee1'].to_numpy(),
                   data['Lessee2'].to_numpy(),
                   data['Lessee3'].to_numpy()])
contract = np.stack([data['ContractType1'].to_numpy(),
                         data['ContractType2'].to_numpy(),
                         data['ContractType3'].to_numpy(),])
if debug:
    print('nbv shape: ', nbv.shape)
    print('cost shape:', cost.shape)
    print('fleetAge shape: ', fleetAge.shape)
    print('status shape:', status.shape)
    print('weightedAge shape:', weightedAge.shape)
    print('productType shape:', product.shape)
    print('lessee shape:', lessee.shape)
    print('contractType shape:', contract.shape)

nbv shape:  (20000,)
cost shape: (20000,)
fleetAge shape:  (3, 20000)
status shape: (20000,)
weightedAge shape: (3, 20000)
productType shape: (3, 20000)
lessee shape: (3, 20000)
contractType shape: (3, 20000)


#### Constraints

TODO: confirm constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. minTotalNbv <= Total NBV <= maxTotalNbv
    
2. minTotalCost <= Total Cost <= maxTotalCost
    
3. Container Age >=
    
4. Status:

    1. OnHire >= OnHireLimit

5. Weighted Age >=

6. Product >=
    
7. Lessee <=
    
8. Contract Type >= 


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem

In [16]:
# variables
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, cat=LpBinary) for i in range(n)])
# problem
prob = LpProblem("MyProblem", LpMaximize if NbvCost else LpMinimize)

In [17]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

In [18]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv is not None:
    prob += lpSum(var * nbv) <= maxTotalNbv
if minTotalNbv is not None:
    prob += lpSum(var * nbv) >= minTotalNbv
    
# cost
if maxTotalCost is not None:
    prob += lpSum(var * cost) <= maxTotalCost
if minTotalCost is not None:
    prob += lpSum(var * cost) >= minTotalCost
    
# container age
if fleetAgeLimit[0] is not None:
    prob += lpSum(var * fleetAge[0]) >= fleetAgeLimit[0] * numSelected
if fleetAgeLimit[1] is not None:
    prob += lpSum(var * fleetAge[1]) >= fleetAgeLimit[1] * numSelected
if fleetAgeLimit[2] is not None:
    prob += lpSum(var * fleetAge[2]) >= fleetAgeLimit[2] * numSelected
    
# status
if OnHireLimit is not None:
    prob += lpSum(var * status) >= OnHireLimit * numSelected
    
# weighted age
if weightedAgeLimit[0] is not None:
    prob += lpSum(var * weightedAge[0]) >= weightedAgeLimit[0] * numSelected
if weightedAgeLimit[1] is not None:
    prob += lpSum(var * weightedAge[1]) >= weightedAgeLimit[1] * numSelected
if weightedAgeLimit[2] is not None:
    prob += lpSum(var * weightedAge[2]) >= weightedAgeLimit[2] * numSelected

# product
if productLimit[0] is not None:
    prob += lpSum(var * product[0]) >= productLimit[0] * numSelected
if productLimit[1] is not None:
    prob += lpSum(var * product[1]) >= productLimit[1] * numSelected
if productLimit[2] is not None:
    prob += lpSum(var * product[2]) >= productLimit[2] * numSelected

# lessee
if lesseeLimit[0] is not None:
    prob += lpSum(var * lessee[0]) <= lesseeLimit[0] * numSelected
if lesseeLimit[1] is not None:
    prob += lpSum(var * lessee[1]) <= lesseeLimit[1] * numSelected
if lesseeLimit[2] is not None:
    prob += lpSum(var * lessee[2]) <= lesseeLimit[2] * numSelected

# contract type
if contractLimit[0] is not None:
    prob += lpSum(var * contract[0]) >= contractLimit[0] * numSelected
if contractLimit[1] is not None:
    prob += lpSum(var * contract[1]) >= contractLimit[1] * numSelected
if contractLimit[2] is not None:
    prob += lpSum(var * contract[2]) >= contractLimit[2] * numSelected

# prob.writeLP('problem.lp')

In [19]:
@timeout_decorator.timeout(TIMEOUT)
def run():
    t_start = time.time()
    prob.solve()
    print('time consumed: ', round(time.time() - t_start, 2), 's')

In [20]:
# solve
try:
    run()
except Exception as e:
    print('Time out in {0} seconds.'.format(TIMEOUT))

time consumed:  3.47 s


In [21]:
print("==============================================================")
# print(prob)
print("status:",LpStatus[prob.status])
print("==============================================================")
print("target value: ",value(prob.objective))

status: Optimal
target value:  64560279.513004474


In [22]:
# if solution is found
if prob.status == 1 or prob.status == 2:
    tmp = prob.variables()
    result = np.array([tmp[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')

17615 / 20000 containers are selected.


### Analysis

For debug only

TODO: if infeasible, it may take time much longer than expected

In [23]:
if debug:
    print("nbv:", sum(result * nbv))
    print("cost:", sum(result * cost))

    print("container age: ")
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[0], fleetAgeUpBound[0]), sum(result * fleetAge[0])/sum(result))
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[1], fleetAgeUpBound[1]), sum(result * fleetAge[1])/sum(result))
    print("\t container age from {0} to {1}:".format(fleetAgeLowBound[2], fleetAgeUpBound[2]), sum(result * fleetAge[2])/sum(result))

    print('billing status:', sum(result * status)/sum(result))

    print("weighted age: ")
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[0], weightedAgeUpBound[0]), sum(result * weightedAge[0])/sum(result))
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[1], weightedAgeUpBound[1]), sum(result * weightedAge[1])/sum(result))
    print("\t weighted age from {0} to {1}:".format(weightedAgeLowBound[2], weightedAgeUpBound[2]), sum(result * weightedAge[2])/sum(result))
    
    print("product: ")
    print("\t product {0}:".format(productType[0]), sum(result * product[0])/sum(result))
    print("\t product {0}:".format(productType[1]), sum(result * product[1])/sum(result))
    print("\t product {0}:".format(productType[2]), sum(result * product[2])/sum(result))
    
    print("lessee: ")
    print("\t lessee {0}:".format(lesseeType[0]), sum(result * lessee[0])/sum(result))
    print("\t lessee {0}:".format(lesseeType[1]), sum(result * lessee[1])/sum(result))
    print("\t lessee {0}:".format(lesseeType[2]), sum(result * lessee[2])/sum(result))
    
    print("contract type: ")
    print("\t contract type {0}:".format(contractType[0]), sum(result * contract[0])/sum(result))
    print("\t contract type {0}:".format(contractType[1]), sum(result * contract[1])/sum(result))
    print("\t contract type {0}:".format(contractType[2]), sum(result * contract[2])/sum(result))
    

nbv: 60784515.33400442
cost: 70317814.43999995
container age: 
	 container age from -inf to 3: 0.34907749077490774
	 container age from 3 to 6: 0.6509225092250922
	 container age from 4 to 10: 0.6420096508657395
billing status: 1.0
weighted age: 
	 weighted age from -inf to 4: 0.34907749077490774
	 weighted age from 3 to 6: 0.21419244961680386
	 weighted age from 4 to 15: 0.6509225092250922
product: 
	 product D20: 0.21419244961680386
	 product D4H: 0.7858075503831962
	 product R4H: 0.0
lessee: 
	 lessee MSC: 0.4228214589838206
	 lessee ONE: 0.3650298041441953
	 lessee HAPAG: 0.19273346579619643
contract type: 
	 contract type LT: 0.5592392847005393
	 contract type LE: 0.24802724950326427
	 contract type LF: 0.19273346579619643


### Output

In [24]:
if prob.status == 1 or prob.status == 2:
    data.drop(['FleetAge1', 'FleetAge2', 'FleetAge3', 'Status', 'WeightedAge1', 'WeightedAge2', 'WeightedAge3', 'ProductType1', 'ProductType2', 'ProductType3', 'Lessee1', 'Lessee2', 'Lessee3', 'ContractType1', 'ContractType2', 'ContractType3'], axis=1, inplace=True)
    data.insert(loc=0, column="selected", value=result)
    data.to_csv('demo_result.csv')

In [25]:
if prob.status == 1 and debug:
    data.sample(3)